# 🎬 Auto-Dubbing Pipeline (Colab GPU)

영어 영상 → 한국어 더빙 + 자막

**사용법:**
1. 런타임 → 런타임 유형 변경 → **GPU (T4)** 선택
2. 셀 순서대로 실행
3. API 키 입력
4. 유튜브 URL 입력

## 1️⃣ 설치

In [ ]:
# 필수 패키지 설치
!pip install -q openai requests yt-dlp torch torchaudio
!apt-get install -y ffmpeg > /dev/null 2>&1

print("✅ 설치 완료!")

## 2️⃣ API 키 설정 ⚠️ 필수!

In [ ]:
#@title API 키 입력 (실행 후 입력창에 키 입력)
import os
from getpass import getpass

print("API 키를 입력하세요:")
print("(입력 시 화면에 표시되지 않습니다)")
print()

openai_key = getpass("OpenAI API Key: ")
elevenlabs_key = getpass("ElevenLabs API Key: ")

os.environ['OPENAI_API_KEY'] = openai_key
os.environ['ELEVENLABS_API_KEY'] = elevenlabs_key

print()
print("✅ API 키 설정 완료!")

## 3️⃣ GPU 확인

In [ ]:
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU 사용 가능: {gpu_name}")
else:
    print("⚠️ GPU 없음! 런타임 → 런타임 유형 변경 → GPU 선택하세요")

## 4️⃣ 더빙 파이프라인 코드

In [ ]:
import os
import json
import subprocess
import tempfile
from pathlib import Path
import torch
import torchaudio
from openai import OpenAI
import requests

# 설정
ELEVENLABS_API_KEY = os.environ.get('ELEVENLABS_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

# ElevenLabs 목소리
VOICES = {
    "male_1": "pNInz6obpgDQGcFmaJgB",    # Adam
    "male_2": "VR6AewLTigWG4xSOukaG",    # Arnold
    "female_1": "pFZP5JQG7iQjIQuC4Bku",  # Lily
    "female_2": "21m00Tcm4TlvDq8ikWAM",  # Rachel
}

def download_youtube(url, output_dir):
    """유튜브 다운로드"""
    print(f"[YouTube] 다운로드 중: {url}")
    cmd = [
        "yt-dlp", "-f", "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]",
        "--merge-output-format", "mp4",
        "-o", f"{output_dir}/%(title)s.%(ext)s",
        url
    ]
    subprocess.run(cmd, check=True)
    files = list(Path(output_dir).glob("*.mp4"))
    if files:
        print(f"[YouTube] 다운로드 완료: {files[0].name}")
        return str(files[0])
    raise Exception("다운로드 실패")

def extract_audio(video_path, audio_path):
    """오디오 추출 (WAV)"""
    cmd = ["ffmpeg", "-y", "-i", video_path, "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", audio_path]
    subprocess.run(cmd, capture_output=True, check=True)
    return audio_path

def detect_speech_vad(audio_path):
    """Silero VAD로 음성 구간 감지"""
    print("[VAD] 음성 구간 감지 중...")
    model, utils = torch.hub.load('snakers4/silero-vad', 'silero_vad', trust_repo=True)
    (get_speech_timestamps, _, read_audio, _, _) = utils
    
    wav = read_audio(audio_path, sampling_rate=16000)
    timestamps = get_speech_timestamps(wav, model, sampling_rate=16000)
    
    segments = []
    for ts in timestamps:
        start = ts['start'] / 16000
        end = ts['end'] / 16000
        segments.append({'start': round(start, 2), 'end': round(end, 2), 'duration': round(end - start, 2)})
    
    print(f"[VAD] {len(segments)}개 음성 구간 감지")
    return segments

def transcribe_segments(audio_path, segments, temp_dir):
    """Whisper로 텍스트 변환"""
    print("[Whisper] 텍스트 변환 중...")
    client = OpenAI()
    
    for i, seg in enumerate(segments):
        print(f"  {i+1}/{len(segments)}", end="\r")
        seg_path = f"{temp_dir}/seg_{i:04d}.mp3"
        cmd = ["ffmpeg", "-y", "-i", audio_path, "-ss", str(seg['start']), "-to", str(seg['end']), "-q:a", "2", seg_path]
        subprocess.run(cmd, capture_output=True)
        
        with open(seg_path, "rb") as f:
            text = client.audio.transcriptions.create(model="whisper-1", file=f, response_format="text")
        seg['text'] = text.strip()
    
    print(f"\n[Whisper] {len(segments)}개 완료")
    return [s for s in segments if s.get('text')]

def translate_segments(segments, tone="formal"):
    """GPT로 번역"""
    print(f"[번역] 번역 중... (말투: {tone})")
    client = OpenAI()
    
    tone_desc = {
        "formal": "존댓말 (~습니다/~요)",
        "casual": "반말 (~해/~야)",
        "narration": "나레이션체 (~다/~했다)"
    }.get(tone, "존댓말")
    
    texts = [{"id": i, "text": s["text"], "duration": s["duration"]} for i, s in enumerate(segments)]
    
    prompt = f"""Translate to Korean ({tone_desc}). Keep translations speakable within the duration.
Return JSON array with "id" and "translation".

{json.dumps(texts, ensure_ascii=False)}"""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    
    content = response.choices[0].message.content
    if "```json" in content:
        content = content.split("```json")[1].split("```")[0]
    elif "```" in content:
        content = content.split("```")[1].split("```")[0]
    
    translations = {item["id"]: item["translation"] for item in json.loads(content.strip())}
    for i, seg in enumerate(segments):
        seg["translated"] = translations.get(i, seg["text"])
    
    print("[번역] 완료")
    return segments

def generate_tts(text, output_path, voice_id):
    """ElevenLabs TTS"""
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
    response = requests.post(url, headers={"xi-api-key": ELEVENLABS_API_KEY, "Content-Type": "application/json"},
                            json={"text": text, "model_id": "eleven_multilingual_v2"})
    response.raise_for_status()
    with open(output_path, "wb") as f:
        f.write(response.content)
    return output_path

def process_tts(segments, temp_dir, voice="female_1"):
    """TTS 생성"""
    print(f"[TTS] 음성 생성 중... (목소리: {voice})")
    voice_id = VOICES.get(voice, VOICES["female_1"])
    
    for i, seg in enumerate(segments):
        print(f"  {i+1}/{len(segments)}", end="\r")
        tts_path = f"{temp_dir}/tts_{i:04d}.mp3"
        generate_tts(seg["translated"], tts_path, voice_id)
        seg["tts_path"] = tts_path
    
    print(f"\n[TTS] {len(segments)}개 완료")
    return segments

def mix_audio(original_audio, segments, output_path, original_volume=0.15):
    """오디오 믹싱"""
    print("[믹싱] 오디오 믹싱 중...")
    
    inputs = ["-i", original_audio]
    filter_parts = [f"[0:a]volume={original_volume}[orig]"]
    overlay_inputs = ["[orig]"]
    
    for i, seg in enumerate(segments):
        inputs.extend(["-i", seg["tts_path"]])
        delay_ms = int(seg["start"] * 1000)
        filter_parts.append(f"[{i+1}:a]adelay={delay_ms}|{delay_ms}[dub{i}]")
        overlay_inputs.append(f"[dub{i}]")
    
    filter_parts.append(f"{''.join(overlay_inputs)}amix=inputs={len(segments)+1}:duration=longest:normalize=0[out]")
    
    cmd = ["ffmpeg", "-y"] + inputs + ["-filter_complex", ";".join(filter_parts), "-map", "[out]", "-c:a", "libmp3lame", output_path]
    subprocess.run(cmd, capture_output=True, check=True)
    print("[믹싱] 완료")
    return output_path

def replace_audio(video_path, audio_path, output_path):
    """영상 오디오 교체"""
    cmd = ["ffmpeg", "-y", "-i", video_path, "-i", audio_path, "-c:v", "copy", "-map", "0:v:0", "-map", "1:a:0", "-shortest", output_path]
    subprocess.run(cmd, capture_output=True, check=True)
    return output_path

def generate_srt(segments, output_path):
    """SRT 자막 생성"""
    def fmt_time(s):
        h, m, sec = int(s//3600), int((s%3600)//60), int(s%60)
        ms = int((s%1)*1000)
        return f"{h:02d}:{m:02d}:{sec:02d},{ms:03d}"
    
    with open(output_path, "w", encoding="utf-8") as f:
        for i, seg in enumerate(segments, 1):
            f.write(f"{i}\n{fmt_time(seg['start'])} --> {fmt_time(seg['end'])}\n{seg['translated']}\n\n")
    return output_path

print("✅ 함수 로드 완료!")

## 5️⃣ 더빙 실행

In [ ]:
#@title 🎬 더빙 설정
youtube_url = "https://www.youtube.com/watch?v=-dtGzB4zqlc" #@param {type:"string"}
voice = "female_1" #@param ["female_1", "female_2", "male_1", "male_2"]
tone = "formal" #@param ["formal", "casual", "narration"]

import time
start_time = time.time()

# 임시 폴더
temp_dir = tempfile.mkdtemp()
print(f"임시 폴더: {temp_dir}")

# 1. 유튜브 다운로드
print("\n" + "="*50)
print("[1/6] 유튜브 다운로드")
video_path = download_youtube(youtube_url, temp_dir)

# 2. 오디오 추출
print("\n" + "="*50)
print("[2/6] 오디오 추출")
audio_wav = f"{temp_dir}/audio.wav"
audio_mp3 = f"{temp_dir}/audio.mp3"
extract_audio(video_path, audio_wav)
subprocess.run(["ffmpeg", "-y", "-i", audio_wav, "-q:a", "2", audio_mp3], capture_output=True)

# 3. VAD
print("\n" + "="*50)
print("[3/6] VAD 음성 감지")
segments = detect_speech_vad(audio_wav)

# 4. Whisper
print("\n" + "="*50)
print("[4/6] Whisper 텍스트 변환")
segments = transcribe_segments(audio_mp3, segments, temp_dir)

# 5. 번역
print("\n" + "="*50)
print("[5/6] 번역")
segments = translate_segments(segments, tone)

# 6. TTS
print("\n" + "="*50)
print("[6/6] TTS 생성")
segments = process_tts(segments, temp_dir, voice)

# 7. 믹싱 & 최종 출력
print("\n" + "="*50)
print("[최종] 합성 중...")
mixed_audio = f"{temp_dir}/mixed.mp3"
mix_audio(audio_mp3, segments, mixed_audio)

output_video = "/content/dubbed_output.mp4"
output_srt = "/content/dubbed_output.srt"
replace_audio(video_path, mixed_audio, output_video)
generate_srt(segments, output_srt)

elapsed = time.time() - start_time
print(f"\n" + "="*50)
print(f"✅ 완료! 소요시간: {int(elapsed//60)}분 {int(elapsed%60)}초")
print(f"📹 영상: {output_video}")
print(f"📝 자막: {output_srt}")

## 6️⃣ 결과 다운로드

In [ ]:
from google.colab import files

# 영상 다운로드
files.download('/content/dubbed_output.mp4')
files.download('/content/dubbed_output.srt')

## 📺 결과 미리보기

In [ ]:
from IPython.display import Video
Video('/content/dubbed_output.mp4', width=640)